

__Set your own route planner for a given OSM map and GTFS file and query server on localhost for a detailed routes__


### OTP walkthrough

from: http://docs.opentripplanner.org/en/latest/Basic-Tutorial/

1. Install Java SDK
2. Download [latest](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/otp-1.4.0-shaded.jar) .jar from [Maven](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/)
3. Download gtfs.zip of a given city into a `folder` from `transit.land`
4. Download osm.pgf of your area from [here](https://www.interline.io/osm/extracts/) (need to login to get API token)
5. call from terminal `java -Xmx6G -jar otp-1.4.0-shaded.jar --build data_path --inMemory` changing `6G` as memory allocated (6GB) and `data_path` to your path with osm and gtfs files
6. if succesful you shall be able to open a web search in your browser: `http://localhost:8080/`
7. and query it via API, for instance: 'http://localhost:8080/otp/routers/default/plan?fromPlace=52.25177%2C20.92758&toPlace=52.20550%2C21.00517&time=1%3A02pm&date=04-09-2020&mode=TRANSIT%2CWALK&maxWalkDistance=5000&arriveBy=false'


In [1]:
import pandas as pd
import requests
import osmnx as ox

local imports

In [2]:
from main import make_query, parse_OTP_response
from utils import plot

set up

In [3]:
PATH = 'requests_10.csv'
OTP_API = "http://localhost:8080/otp/routers/default/plan"

In [4]:
df = pd.read_csv(PATH)
G = ox.load_graphml('Krakow.graphml')
df['origin_y'] = df.apply(lambda row: G.nodes[row.origin]['y'], axis=1)
df['origin_x'] = df.apply(lambda row: G.nodes[row.origin]['x'], axis=1)
df['destination_y'] = df.apply(lambda row: G.nodes[row.destination]['y'], axis=1)
df['destination_x'] = df.apply(lambda row: G.nodes[row.destination]['x'], axis=1)
df.treq = pd.to_datetime(df.treq)
df.sample(5)[['origin_x','origin_y','destination_x','destination_y','treq']]

,origin_x,origin_y,destination_x,destination_y,treq
1857,19.874259,50.014862,19.965809,50.064086,2024-03-27 08:00:17
2742,19.864756,50.010229,19.938712,50.046949,2024-03-27 08:07:36
3480,19.875866,50.012092,19.958617,50.058002,2024-03-27 08:13:03
2834,19.869984,50.011822,19.949797,50.057791,2024-03-27 08:08:17
1215,19.875955,50.016279,19.971044,50.044166,2024-03-27 07:55:20


input

In [13]:
#when use KRK not need this part!!!!
# df = pd.read_csv(PATH, index_col=[0])  # load the csv
# df.treq = pd.to_datetime(df.treq)
# df.sample(3)[['origin_x','origin_y','destination_x','destination_y','treq']]

In [14]:
#!!! check for Poznan
# df1 = pd.DataFrame(columns=['origin_x','origin_y','destination_x','destination_y','treq'])
# df1['origin_x'] = [16.89423, 16.91294, 16.94984]
# df1['origin_y'] = [52.40504, 52.42797, 52.41404]

# df1['destination_x'] = [16.94984, 16.92324, 16.88410]
# df1['destination_y'] = [52.40619, 52.39833, 52.36973]

# df1['treq'] = pd.Timestamp(2023, 3, 7, 12)

# df =df1

sample query (before you can query the server you need to run it -see run OTP server.ipynb in this folder)

[query synthax](http://dev.opentripplanner.org/apidoc/1.0.0/resource_PlannerResource.html)  
[response json structure](http://dev.opentripplanner.org/apidoc/1.0.0/json_Response.html)

In [7]:
row = df.sample(1).squeeze()
query = make_query(row)
r = requests.get(OTP_API, params=query)
route = parse_OTP_response(r.json())
route

{'success': True,
 'n_itineraries': 3,
 'duration': 2688,
 'walkDistance': 431.4195639073362,
 'transfers': 1,
 'transitTime': 2160,
 'waitingTime': 182,
 'modes': [['WALK', 210, 268],
  ['BUS', 1380, 6130],
  ['BUS', 780, 4327],
  ['WALK', 136, 163]]}

#### visualization

In [8]:
plot(r.json()['plan'], color = 'green')

Trip from (19.8727,50.0159) to (19.9818,50.0491) at 1711522980000. 
3 connections found. 
Best one is 45min (431m walk, 1 transfer(s), wait time 3.03min)

LEG 	 MODE 	DIST 	TIME
-----------------------------
1	WALK	268	210
2	BUS	6130	1380
3	BUS	4327	780
4	WALK	163	136


output

In [12]:
def merge_batches(path, out_path, remove=True):
    """
    walks through directory and merges all csv files into one
    :param path:
    :param out_path:
    :param remove:
    :return: none
    """

    all_files = glob.glob(path + "/*.csv")
    df = pd.concat((pd.read_csv(f) for f in all_files), sort=False).set_index('id')
    df.to_csv(os.path.join(out_path))
    if remove:
        for f in all_files:
            os.remove(f)

In [ ]:
# pd.read_csv('georequests_PT.csv', index_col=[0]).sample(5)

full response

In [ ]:
# r.json()['requestParameters']

---